In [20]:
#principale missions : detect image 
#undestand gesture 
#provide ways to correct it 
# mission 1 classify gesture from the four principale exercices ---> add data 
# Squat , Push Up , Planck,Burpees

# Part One : Workout Exercice classification 

In [14]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
types = os.listdir('C:\\Users\\km_ka\\Desktop\\workout\\train')
print (types)  

['plank', 'push-up', 'squat']


### Data preparation

In [15]:
items = []
for item in types:
 # Get all the file names
 sub_files = os.listdir('C:\\Users\\km_ka\\Desktop\\workout\\train' + '\\' +item)
 # Add them to the list
 for exe in sub_files:
    items.append((item, str('C:\\Users\\km_ka\\Desktop\\workout\\train' + '\\' +item) + '\\' + exe))
train_df = pd.DataFrame(data=items, columns=['exercice_type', 'video_name'])
print(train_df.head())
print(train_df.tail())
df = train_df.loc[:,['video_name','exercice_type']]
df.to_csv('train.csv')

  exercice_type                                         video_name
0         plank  C:\Users\km_ka\Desktop\workout\train\plank\pla...
1         plank  C:\Users\km_ka\Desktop\workout\train\plank\pla...
2         plank  C:\Users\km_ka\Desktop\workout\train\plank\pla...
3         plank  C:\Users\km_ka\Desktop\workout\train\plank\pla...
4         plank  C:\Users\km_ka\Desktop\workout\train\plank\pla...
   exercice_type                                         video_name
70         squat  C:\Users\km_ka\Desktop\workout\train\squat\squ...
71         squat  C:\Users\km_ka\Desktop\workout\train\squat\squ...
72         squat  C:\Users\km_ka\Desktop\workout\train\squat\squ...
73         squat  C:\Users\km_ka\Desktop\workout\train\squat\squ...
74         squat  C:\Users\km_ka\Desktop\workout\train\squat\squ...


In [16]:
from tensorflow_docs.vis import embed
from tensorflow import keras
import matplotlib.pyplot as plt
import tensorflow as tf

In [17]:
train_df = pd.read_csv("train.csv")
print(f"Total videos for training: {len(train_df)}")
train_df.sample(10)

Total videos for training: 75


,Unnamed: 0,video_name,exercice_type
45,45,C:\Users\km_ka\Desktop\workout\train\push-up\p...,push-up
35,35,C:\Users\km_ka\Desktop\workout\train\push-up\p...,push-up
57,57,C:\Users\km_ka\Desktop\workout\train\squat\squ...,squat
56,56,C:\Users\km_ka\Desktop\workout\train\squat\squ...,squat
3,3,C:\Users\km_ka\Desktop\workout\train\plank\pla...,plank
51,51,C:\Users\km_ka\Desktop\workout\train\squat\squ...,squat
7,7,C:\Users\km_ka\Desktop\workout\train\push-up\p...,push-up
15,15,C:\Users\km_ka\Desktop\workout\train\push-up\p...,push-up
52,52,C:\Users\km_ka\Desktop\workout\train\squat\squ...,squat
60,60,C:\Users\km_ka\Desktop\workout\train\squat\squ...,squat


In [22]:
import cv2
IMG_SIZE = 224
def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)
            if len(frames) == max_frames:
             break
    finally:
        cap.release()
    return np.array(frames)


In [23]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")

feature_extractor = build_feature_extractor()

87910968/87910968 [==============================] - 83s 1us/step


In [32]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["exercice_type"]))
print(label_processor.get_vocabulary())
labels = train_df["exercice_type"].values
labels = label_processor(labels[..., None]).numpy()
labels
print(train_df.head())


['plank', 'push-up', 'squat']
   Unnamed: 0                                         video_name exercice_type
0           0  C:\Users\km_ka\Desktop\workout\train\plank\pla...         plank
1           1  C:\Users\km_ka\Desktop\workout\train\plank\pla...         plank
2           2  C:\Users\km_ka\Desktop\workout\train\plank\pla...         plank
3           3  C:\Users\km_ka\Desktop\workout\train\plank\pla...         plank
4           4  C:\Users\km_ka\Desktop\workout\train\plank\pla...         plank


In [34]:
BATCH_SIZE = 64
EPOCHS = 100

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

In [35]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    
    labels = df["exercice_type"].values
    labels = label_processor(labels[..., None]).numpy()

    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool") # 145,20
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32") #145,20,2048

    for idx, path in enumerate(video_paths):
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "train")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")
print(f"train_labels in train set: {train_labels.shape}")





1/1 [==============================] - 0s 156ms/step


1/1 [==============================] - 0s 155ms/step


1/1 [==============================] - 0s 283ms/step


1/1 [==============================] - 0s 159ms/step


1/1 [==============================] - 0s 208ms/step


1/1 [==============================] - 0s 188ms/step


1/1 [==============================] - 0s 150ms/step


1/1 [==============================] - 0s 147ms/step


1/1 [==============================] - 0s 163ms/step


1/1 [==============================] - 0s 169ms/step
Frame features in train set: (75, 20, 2048)
Frame masks in train set: (75, 20)
train_labels in train set: (75, 1)


In [43]:
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")
    x = keras.layers.GRU(16, return_sequences=True)(frame_features_input, mask=mask_input)
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

EPOCHS = 30
def run_experiment():
    filepath = "./tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )
    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()
   

Epoch 1/30
2/2 [==============================] - ETA: 0s - loss: 1.3008 - accuracy: 0.2115 
Epoch 1: val_loss improved from inf to 1.29309, saving model to ./tmp\video_classifier
2/2 [==============================] - 38s 9s/step - loss: 1.3008 - accuracy: 0.2115 - val_loss: 1.2931 - val_accuracy: 0.0000e+00
Epoch 2/30
2/2 [==============================] - ETA: 0s - loss: 0.8896 - accuracy: 0.7500
Epoch 2: val_loss did not improve from 1.29309
2/2 [==============================] - 2s 1s/step - loss: 0.8896 - accuracy: 0.7500 - val_loss: 1.3661 - val_accuracy: 0.0000e+00
Epoch 3/30
2/2 [==============================] - ETA: 0s - loss: 0.9355 - accuracy: 0.6346
Epoch 3: val_loss did not improve from 1.29309
2/2 [==============================] - 2s 1s/step - loss: 0.9355 - accuracy: 0.6346 - val_loss: 1.3929 - val_accuracy: 0.0000e+00
Epoch 4/30
2/2 [==============================] - ETA: 0s - loss: 0.9063 - accuracy: 0.7115
Epoch 4: val_loss did not improve from 1.29309
2/2 [=======

1/1 [==============================] - 0s 450ms/step - loss: 0.8600 - accuracy: 0.8095
Test accuracy: 80.95%


In [39]:
dataset_path = os.listdir('C:\\Users\\km_ka\\Desktop\\workout\\test')
print(dataset_path)

room_types = os.listdir('C:\\Users\\km_ka\\Desktop\\workout\\test')
print("Types of activities found: ", len(dataset_path))

rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir('C:\\Users\\km_ka\\Desktop\\workout\\test' + '\\' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str('C:\\Users\\km_ka\\Desktop\\workout\\test' + '\\' +item) + '\\' + room))
    
# Build a dataframe        
test_df = pd.DataFrame(data=rooms, columns=['exercice_type', 'video_name'])
print(test_df.head())
print(test_df.tail())

df = test_df.loc[:,['video_name','exercice_type']]
df
df.to_csv('test.csv')

['plank', 'push-up', 'squat']
Types of activities found:  3
  exercice_type                                         video_name
0         plank  C:\Users\km_ka\Desktop\workout\test\plank\pexe...
1         plank  C:\Users\km_ka\Desktop\workout\test\plank\prod...
2       push-up  C:\Users\km_ka\Desktop\workout\test\push-up\pu...
3       push-up  C:\Users\km_ka\Desktop\workout\test\push-up\pu...
4       push-up  C:\Users\km_ka\Desktop\workout\test\push-up\pu...
   exercice_type                                         video_name
16       push-up  C:\Users\km_ka\Desktop\workout\test\push-up\pu...
17       push-up  C:\Users\km_ka\Desktop\workout\test\push-up\pu...
18       push-up  C:\Users\km_ka\Desktop\workout\test\push-up\pu...
19         squat  C:\Users\km_ka\Desktop\workout\test\squat\pexe...
20         squat  C:\Users\km_ka\Desktop\workout\test\squat\pexe...


In [40]:
test_df = pd.read_csv("test.csv")
print(f"Total videos for testing: {len(test_df)}")


Total videos for testing: 21


In [41]:
test_data, test_labels = prepare_all_videos(test_df, "test")
print(f"test_labels in train set: {test_labels.shape}")

1/1 [==============================] - 0s 174ms/step


1/1 [==============================] - 0s 253ms/step


1/1 [==============================] - 0s 216ms/step
test_labels in train set: (21, 1)


In [48]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames

test_video = r'C:\Users\km_ka\Desktop\workout\test\squat\pexels-mart-production-8837221 (2160p).mp4'
print(f"Test video path: {test_video}")

test_frames = sequence_prediction(test_video)


Test video path: C:\Users\km_ka\Desktop\workout\test\squat\pexels-mart-production-8837221 (2160p).mp4
1/1 [==============================] - 0s 42ms/step
  push-up: 47.74%
  plank: 26.17%
  squat: 26.10%


In [62]:
from IPython.display import HTML
HTML("""
    <video alt="test" width="520" height="440" controls>
        <source src='C:/Users/km_ka/Desktop/workout/test/squat/pexels-mart-production-8837221 (2160p).mp4' type="video/mp4" style="height:300px;width:300px">
    </video>
""")

# Part two : tracking poses


### VIDEO FEED

In [66]:
import cv2
import mediapipe as mp 
import numpy as np

In [70]:
drawing=mp.solutions.drawing_utils
pose=mp.solutions.pose

In [72]:
cap=cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read();
    cv2.imshow('Mediapipe Feed', frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

### Make detection

In [108]:
def calculate_angle (a,b,c):
    a=np.array(a)
    b=np.array(b)
    c=np.array(c)
    radian=np.arctan2(c[1]-b[1],c[0]-b[0])-np.arctan2(a[1]-b[1],a[0]-b[0]);
    angle=np.abs(radian*180.0/np.pi);
    if angle>180.0:
        angle=360-angle;
    return(angle)

    

In [112]:
cap=cv2.VideoCapture(0)
with pose.Pose(min_detection_confidence = 0.5,min_tracking_confidence=0.5) as poset:
  while cap.isOpened():
    ret, frame = cap.read();
    #recoloring image 
    image=cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
   
    #Make detection 
    results = poset.process(image)
    #Recolor back to BRG

    try: 
        landmarks=results.pose_landmarks.landmark
        shoulders=[landmarks[pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[pose.PoseLandmark.LEFT_SHOULDER.value].y]
        elbow=[landmarks[pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[pose.PoseLandmark.LEFT_ELBOW.value].y]
        wrist=[landmarks[pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[pose.PoseLandmark.LEFT_WRIST.value].y]
        arm_angle=calculate_angle(shoulders,elbow,wrist)
        print(arm_angle);
        cv2.putText(image, str(arm_angle), tuple(np.multiply(elbow, [480, 640]).astype(int)),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

    except:
        pass
    #detection rendering 
    drawing.draw_landmarks(image, results.pose_landmarks , pose.POSE_CONNECTIONS)
    cv2.imshow('Mediapipe Feed', image)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

165.25207063759504
170.36623545693723
108.10004216141293
46.64868305238538
29.934722561813704
1.5198092197564972
0.9044251946206201
2.679926788867989
4.138039885732436
5.778701553983404
6.866429853959639
5.982094466779802
5.975428010714193
5.094010765527089
4.602708376782771
5.991807768349461
4.979308401775194
0.564174304470766
1.6343131120770917
1.0153593839812118
0.14272589955658885
1.0600774519279759
1.2018823370174014
8.284711881370578
3.6132838171666664
2.935114542218508
4.000899512235097
5.704980232192309
9.690386980153592
10.676557764955776
9.37333852582918
15.179965082926856
11.329308108658836
10.66056733449904
13.370893204765066
14.611358464960578
15.205481296909209
16.085382144661693
18.564328620950178
16.202392583843384
18.074934749942674
17.885046906731038
18.070210539006997
10.384694750440582
8.776728807957364
9.125138268135927
5.1918636650117795
2.761669634506044
0.856478286049625
2.113318277698528
4.98480870153352
3.8315261751598833
4.887470253397689
2.388741515096861
0.

179.65985440384827
179.50788530793056
179.9675043835342
179.50647843030706
173.70550168042487
166.30936852581925
160.28004321844207
29.426092508500933
6.674056451740695
2.6722621416518546
1.4657454742654774
1.3399036552225496
0.1702549830476998
2.176785341828653
2.9238673546068297
4.5066765420493375
4.1667649488601946
4.808075889476749
6.929130382303903
16.972980431384574
164.99068476258066
164.86739553587714
163.3724658500999
166.32731492439495
165.68238215453326
166.05386003751462
167.62744720825762
168.03797895126328
166.92674114804342
163.38502080740503
161.82513468548308
163.02062915356063
167.45738659199301
168.25957140073658
169.218137607996
169.23249506317643
168.97085874076495
168.39368946545088
167.84860407152442
167.33218090117742
169.2966812421795
171.7619689043916
172.653918959893
172.08136389940572
171.86978648604227
171.59732841522006
172.21211902893728
172.30703198259468
171.01925286999278
170.31913225404762
171.62313995862058
171.30880158774377
172.3418496655254
174.02

In [93]:
len(landmarks)

33

In [94]:
for landmrk in pose.PoseLandmark:
    print(landmrk)

PoseLandmark.NOSE
PoseLandmark.LEFT_EYE_INNER
PoseLandmark.LEFT_EYE
PoseLandmark.LEFT_EYE_OUTER
PoseLandmark.RIGHT_EYE_INNER
PoseLandmark.RIGHT_EYE
PoseLandmark.RIGHT_EYE_OUTER
PoseLandmark.LEFT_EAR
PoseLandmark.RIGHT_EAR
PoseLandmark.MOUTH_LEFT
PoseLandmark.MOUTH_RIGHT
PoseLandmark.LEFT_SHOULDER
PoseLandmark.RIGHT_SHOULDER
PoseLandmark.LEFT_ELBOW
PoseLandmark.RIGHT_ELBOW
PoseLandmark.LEFT_WRIST
PoseLandmark.RIGHT_WRIST
PoseLandmark.LEFT_PINKY
PoseLandmark.RIGHT_PINKY
PoseLandmark.LEFT_INDEX
PoseLandmark.RIGHT_INDEX
PoseLandmark.LEFT_THUMB
PoseLandmark.RIGHT_THUMB
PoseLandmark.LEFT_HIP
PoseLandmark.RIGHT_HIP
PoseLandmark.LEFT_KNEE
PoseLandmark.RIGHT_KNEE
PoseLandmark.LEFT_ANKLE
PoseLandmark.RIGHT_ANKLE
PoseLandmark.LEFT_HEEL
PoseLandmark.RIGHT_HEEL
PoseLandmark.LEFT_FOOT_INDEX
PoseLandmark.RIGHT_FOOT_INDEX
